In [109]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern

def analyze_drain_robust_background_removal(image_path, use_roi=True): # use_roi=True로 기본값 설정
    # 1) HSV/BGR thresholds (완전 까만 빈 부분 제외에 초점)
    brown_lower = np.array([0,  20,  50])   # V 10에서 50으로 올림
    brown_upper = np.array([70, 255, 255])
    
    gray_lower  = np.array([0,   40,  80])   # S 10에서 40으로, V 30에서 80으로 대폭 상향
    gray_upper  = np.array([180,100,200])    # V 상한 유지

    pack_lower  = np.array([0,   80,  50])
    pack_upper  = np.array([180,255,255])
    
    green_lower = np.array([30, 20, 10])
    green_upper = np.array([90, 255, 255])

    white_lower_bgr = np.array([100,100,100]) # 60에서 100으로 다시 올림
    white_upper_bgr = np.array([255,255,255])

    hsv_white_lower = np.array([0, 0, 100])     # V 80에서 100으로 올림
    hsv_white_upper = np.array([180, 50, 255])


    # 2) Load image
    full_image = cv2.imread(image_path)
    if full_image is None:
        print(f"Cannot load image: {image_path}")
        return None

    h_full, w_full = full_image.shape[:2]
    
    # --- ROI 적용 여부 결정 (70% ROI) ---
    if use_roi:
        roi_width = int(w_full * 0.7)  # 너비 70%
        roi_height = int(h_full * 0.7) # 높이 70%
        x_start = int((w_full - roi_width) / 2)
        y_start = int((h_full - roi_height) / 2)
        roi = full_image[y_start : y_start + roi_height, x_start : x_start + roi_width]
        if roi.size == 0:
            print(f"ROI is empty for image: {image_path}. Check ROI dimensions.")
            return None
    else:
        roi = full_image.copy()
        x_start, y_start = 0, 0 
        roi_width, roi_height = w_full, h_full

    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    # 3) Build masks
    mask_brown = cv2.inRange(hsv, brown_lower, brown_upper)
    mask_gray  = cv2.inRange(hsv, gray_lower,  gray_upper)
    soil_mask  = cv2.bitwise_or(mask_brown, mask_gray)
    soil_mask = cv2.morphologyEx(soil_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=1)
    soil_mask = cv2.morphologyEx(soil_mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8), iterations=1)

    pack_mask  = cv2.inRange(hsv, pack_lower, pack_upper)
    pack_mask  = cv2.bitwise_and(pack_mask, cv2.bitwise_not(soil_mask)) 
    pack_mask = cv2.morphologyEx(pack_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=1)

    green_mask = cv2.inRange(hsv, green_lower, green_upper)
    green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=1)

    white_mask_bgr = cv2.inRange(roi, white_lower_bgr, white_upper_bgr) 
    white_mask_hsv = cv2.inRange(hsv, hsv_white_lower, hsv_white_upper)
    
    white_candidates_mask = cv2.bitwise_or(white_mask_bgr, white_mask_hsv)

    white_candidates_clean = cv2.morphologyEx(white_candidates_mask,
                                              cv2.MORPH_OPEN,
                                              np.ones((2,2), np.uint8)) 
    
    contours_white, _ = cv2.findContours(white_candidates_clean,
                                         cv2.RETR_EXTERNAL,
                                         cv2.CHAIN_APPROX_SIMPLE)
    
    cig_mask = np.zeros_like(white_candidates_clean) 

    for c in contours_white:
        x,y,w,h = cv2.boundingRect(c)
        area = cv2.contourArea(c)

        if (h > w * 2.5 and area > 80) or (w > h * 2.5 and area > 80): # 담배 형태 필터링
            cv2.drawContours(cig_mask, [c], -1, 255, cv2.FILLED)
        elif area > 400 and h/w < 2 and w/h < 2: # 휴지 등
            cv2.drawContours(cig_mask, [c], -1, 255, cv2.FILLED)


    lbp_image = local_binary_pattern(gray, P=8, R=1, method='uniform')
    
    lbp_threshold = 60 # LBP 임계값 (빈 공간/그림자 질감 확실히 제외)
    lbp_mask = (lbp_image > lbp_threshold).astype(np.uint8) * 255
    lbp_mask = cv2.morphologyEx(lbp_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8), iterations=1)
    lbp_mask = cv2.morphologyEx(lbp_mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8), iterations=1)


    combined_initial_mask = cv2.bitwise_or(soil_mask, pack_mask)
    combined_initial_mask = cv2.bitwise_or(combined_initial_mask, green_mask)
    combined_initial_mask = cv2.bitwise_or(combined_initial_mask, cig_mask) 
    combined_initial_mask = cv2.bitwise_or(combined_initial_mask, lbp_mask) 

    kernel_small = np.ones((3,3), np.uint8)
    kernel_medium = np.ones((5,5), np.uint8)
    
    processed_mask = cv2.morphologyEx(combined_initial_mask, cv2.MORPH_OPEN, kernel_small, iterations=1)
    processed_mask = cv2.morphologyEx(processed_mask, cv2.MORPH_CLOSE, kernel_medium, iterations=1) # 최종 Closing 강도 약화
    processed_mask = cv2.morphologyEx(processed_mask, cv2.MORPH_OPEN, kernel_small, iterations=1)
    
    final_contamination_mask = processed_mask

    # --- 격자 제외 로직 (ROI 내에서 작동) ---
    blurred_gray = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blurred_gray, 50, 150) # Canny 임계값 유지

    # 디버깅용: Canny 엣지 시각화 (ROI 이미지 기준)
    # cv2.imshow("Canny Edges (ROI Grid)", edges)
    # cv2.waitKey(0)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    grid_mask = np.zeros_like(gray, dtype=np.uint8)
    # ROI 내의 이미지 면적을 기준으로 면적 필터링
    # ROI 70% 설정에 맞춰 격자 칸 면적 범위 재조정 필요 (이전 500~2000에서 100~1000 정도로)
    # 2025-06-09 172528.jpg 이미지의 격자 칸 크기를 보고 조정
    current_area = roi.shape[:2][0] * roi.shape[:2][1] 
    
    for c in contours:
        area = cv2.contourArea(c)
        if area < 100 or area > 1000: # 이 값은 70% ROI 내의 격자 칸에 맞춰 재조정 필요
            continue
        
        epsilon = 0.04 * cv2.arcLength(c, True) 
        approx = cv2.approxPolyDP(c, epsilon, True)

        if len(approx) >= 4 and len(approx) <= 6: 
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w)/h
            
            if aspect_ratio >= 0.7 and aspect_ratio <= 1.3: 
                cv2.rectangle(grid_mask, (x,y), (x+w, y+h), 255, cv2.FILLED)

    grid_mask = cv2.morphologyEx(grid_mask, cv2.MORPH_CLOSE, np.ones((10,10), np.uint8), iterations=2) # 커널 크기 약간 줄임
    grid_mask = cv2.morphologyEx(grid_mask, cv2.MORPH_OPEN, np.ones((7,7), np.uint8), iterations=1)

    # 디버깅용: 최종 Grid Mask 시각화 (ROI 이미지 기준)
    # cv2.imshow("Final Grid Mask (ROI)", grid_mask)
    # cv2.waitKey(0)

    final_contamination_mask = cv2.bitwise_and(final_contamination_mask, cv2.bitwise_not(grid_mask))

    # 4) Compute combined ratio
    total_pixels_analyzed = roi.shape[:2][0] * roi.shape[:2][1]
    contaminated_pixels_analyzed = cv2.countNonZero(final_contamination_mask)
    
    if total_pixels_analyzed > 0:
        combined_ratio = (contaminated_pixels_analyzed / total_pixels_analyzed) * 100
    else:
        combined_ratio = 0.0

    print(f"Combined contamination ratio ({'ROI' if use_roi else 'Full Image'}): {combined_ratio:.1f}%")

    # --- 결과 시각화 (원본 이미지에 ROI 마스크 오버레이) ---
    final_contamination_mask_full = np.zeros_like(full_image[:,:,0], dtype=np.uint8)
    final_contamination_mask_full[y_start : y_start + roi_height, x_start : x_start + roi_width] = final_contamination_mask

    overlay = full_image.copy()
    color_mask = np.zeros_like(full_image, dtype=np.uint8)
    color_mask[final_contamination_mask_full == 255] = [0, 0, 255]

    alpha = 0.5 
    result_image_full = cv2.addWeighted(overlay, 1 - alpha, color_mask, alpha, 0)
    
    display_width = 800 
    display_height = int(full_image.shape[:2][0] * (display_width / full_image.shape[:2][1]))
    
    resized_original = cv2.resize(full_image, (display_width, display_height), interpolation=cv2.INTER_AREA)
    resized_result = cv2.resize(result_image_full, (display_width, display_height), interpolation=cv2.INTER_AREA)

    combined_display_image = np.hstack((resized_original, resized_result))

    cv2.imshow(f"Original vs Contamination ({combined_ratio:.1f}%)", combined_display_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return combined_ratio

# --- 함수 호출 예시 ---
# 이 이미지의 경우, 배경 오탐이 매우 심각하므로 ROI 사용 여부와 무관하게 임계값 조정이 필요합니다.
# 하지만 요청에 따라 use_roi=True로 설정하고 70% ROI에 맞춰 튜닝을 시도합니다.
image_path = "C:/Users/USER/Pictures/Screenshots/2025-06-10 112402.png"
contamination_percentage = analyze_drain_robust_background_removal(image_path, use_roi=True) # use_roi=True로 변경
if contamination_percentage is not None:
    print(f"{image_path}의 ROI (70%) 오염 영역은 약 {contamination_percentage:.1f}% 입니다.")

Combined contamination ratio (ROI): 43.1%
C:/Users/USER/Pictures/Screenshots/2025-06-10 112402.png의 ROI (70%) 오염 영역은 약 43.1% 입니다.
